## Data Preparation

In [3]:
!pip list

Package                       Version
----------------------------- ---------
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bitarray                      2.5.1
bkcharts                      0.2
black          

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate


ImportError: DLL load failed while importing _ufuncs: No se encontró el proceso especificado.

In [24]:
movies_data_path = '../dataset/movies.csv'
finantial_data_path = '../dataset/finantials.csv'
opening_data_path = '../dataset/opening_gross.csv'

In [3]:
fin_data = pd.read_csv(finantial_data_path)
movie_data = pd.read_csv(movies_data_path)
opening_data = pd.read_csv(opening_data_path)

In [28]:
opening_data

,movie_title,opening_gross,screens
0,10 Days in a Madhouse,2451.0,10.0
1,10 Things I Hate About You,8330681.0,2271.0
2,102 Dalmatians,19883351.0,2704.0
3,12 Rounds,5329240.0,2331.0
4,12 Years a Slave,923715.0,19.0
...,...,...,...
2267,Zombieland,24733155.0,3036.0
2268,Zookeeper,20065617.0,3482.0
2269,Zoolander 2,15650000.0,3394.0
2270,Zoom,4510408.0,NaN


In [4]:
numeric_columns_mask = (movie_data.dtypes == float) | (movie_data.dtypes == int)
numeric_columns = [column for column in numeric_columns_mask.index if numeric_columns_mask[column]]
movie_data = movie_data[numeric_columns+['movie_title']]

In [5]:
fin_data = fin_data[['movie_title','production_budget','worldwide_gross']]

In [6]:
fin_movie_data = pd.merge(fin_data, movie_data, on= 'movie_title', how='left')

In [7]:
full_movie_data = pd.merge( opening_data,fin_movie_data, on = 'movie_title', how='left')

In [8]:
full_movie_data[(full_movie_data.worldwide_gross != 0) & (full_movie_data.worldwide_gross.notnull())].shape

(2304, 11)

In [9]:
full_movie_data = full_movie_data.drop(['movie_title','gross'],axis=1)

In [10]:
full_movie_data.columns

Index(['opening_gross', 'screens', 'production_budget', 'worldwide_gross',
       'title_year', 'aspect_ratio', 'duration', 'budget', 'imdb_score'],
      dtype='object')

## Modeling

In [17]:
!pip install scikit-learn

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate
import numpy as np

ImportError: DLL load failed while importing _ufuncs: No se encontró el proceso especificado.

In [ ]:
X = full_movie_data.drop(['worldwide_gross'], axis = 1)
y = full_movie_data['worldwide_gross']

In [13]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [14]:
results = cross_validate(pipeline ,X,y,return_train_score=True,cv=10)
results

{'fit_time': array([0.33126116, 0.32594347, 0.32048607, 0.30208063, 0.32461882,
        0.32264709, 0.31112003, 0.31348014, 0.30392337, 0.30737305]),
 'score_time': array([0.00176954, 0.00277519, 0.00225043, 0.00226712, 0.00200582,
        0.00266671, 0.00191855, 0.00207758, 0.00160885, 0.00166345]),
 'test_score': array([0.67386087, 0.84945318, 0.64398525, 0.77411956, 0.78347799,
        0.85577395, 0.76009985, 0.86835663, 0.65362997, 0.65825827]),
 'train_score': array([0.91673951, 0.91581777, 0.9228721 , 0.91654412, 0.92172829,
        0.91476722, 0.92151444, 0.91734995, 0.92320705, 0.91766026])}

In [15]:
train_score = np.mean(results['train_score'])
test_score = np.mean(results['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.9188200721676859
Test Score: 0.7521015504747262


## Hyperparameter tunning

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
param_tunning = {'core_model__n_estimators': range(20,501,20)} 

In [19]:
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [20]:
grid_search= GridSearchCV(estimator,
                       param_grid = param_tunning,
                       scoring='r2',
                       cv=5) 

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35,random_state= 42)

In [22]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('core_model',
                                        GradientBoostingRegressor())]),
             param_grid={'core_model__n_estimators': range(20, 501, 20)},
             scoring='r2')

In [23]:
final_result = cross_validate(grid_search.best_estimator_,X_train,y_train,return_train_score=True,cv=7)

In [24]:
train_score = np.mean(final_result['train_score'])
test_score = np.mean(final_result['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.9667702920281013
Test Score: 0.7612924726750518


In [25]:
grid_search.best_estimator_.get_params()

{'memory': None,
 'steps': [('imputer', SimpleImputer()),
  ('core_model', GradientBoostingRegressor(n_estimators=220))],
 'verbose': False,
 'imputer': SimpleImputer(),
 'core_model': GradientBoostingRegressor(n_estimators=220),
 'imputer__add_indicator': False,
 'imputer__copy': True,
 'imputer__fill_value': None,
 'imputer__missing_values': nan,
 'imputer__strategy': 'mean',
 'imputer__verbose': 0,
 'core_model__alpha': 0.9,
 'core_model__ccp_alpha': 0.0,
 'core_model__criterion': 'friedman_mse',
 'core_model__init': None,
 'core_model__learning_rate': 0.1,
 'core_model__loss': 'squared_error',
 'core_model__max_depth': 3,
 'core_model__max_features': None,
 'core_model__max_leaf_nodes': None,
 'core_model__min_impurity_decrease': 0.0,
 'core_model__min_samples_leaf': 1,
 'core_model__min_samples_split': 2,
 'core_model__min_weight_fraction_leaf': 0.0,
 'core_model__n_estimators': 220,
 'core_model__n_iter_no_change': None,
 'core_model__random_state': None,
 'core_model__subsample'

In [26]:
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor(n_estimators=220,
                                             alpha=0.9,
                                             ccp_alpha=0.0,
                                             criterion='friedman_mse',
                                             init=None,
                                             learning_rate=0.1,
                                             loss='squared_error',
                                             max_depth=3,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_iter_no_change=None,
                                             random_state=None,
                                             subsample=1.0,
                                             tol=0.0001,
                                             validation_fraction=0.1,
                                             verbose=0,
                                             warm_start=False))
])

In [27]:
estimator.fit(X_train,y_train)

Pipeline(steps=[('imputer', SimpleImputer()),
                ('core_model', GradientBoostingRegressor(n_estimators=220))])

In [28]:
estimator.score(X_test, y_test)

0.7297050595008379

## Saving model

In [14]:
from joblib import dump
from sklearn.externals import joblib

ImportError: DLL load failed while importing _ufuncs: No se encontró el proceso especificado.

In [13]:
print(joblib.__version__)

NameError: name 'joblib' is not defined

In [9]:
joblib.dump(estimator, '../model/model.pkl')

NameError: name 'joblib' is not defined

In [31]:
X_train.columns

Index(['opening_gross', 'screens', 'production_budget', 'title_year',
       'aspect_ratio', 'duration', 'cast_total_facebook_likes', 'budget',
       'imdb_score'],
      dtype='object')

In [2]:
json_pred = {"opening_gross": 17435092, "screens": 3008, "production_budget": 65000000, "title_year": 2012, "aspect_ratio": 2.35, "duration": 99, "cast_total_facebook_likes": 1375, "budget": 65000000, "imdb_score": 6.4}
json_pred

{'opening_gross': 17435092,
 'screens': 3008,
 'production_budget': 65000000,
 'title_year': 2012,
 'aspect_ratio': 2.35,
 'duration': 99,
 'cast_total_facebook_likes': 1375,
 'budget': 65000000,
 'imdb_score': 6.4}

In [7]:
import pandas as pd

# Convertir la lista de python en una lista de tuplas
# tuples = [(key, val) for key, val in json_pred.items()]

# Crear un DataFrame a partir de la lista de tuplas
df = pd.DataFrame([json_pred])

# Convertir el DataFrame en formato json
# json_result = df.to_json(orient='records')[1:-1].replace('},{', '} {')

In [8]:
df

,opening_gross,screens,production_budget,title_year,aspect_ratio,duration,cast_total_facebook_likes,budget,imdb_score
0,17435092,3008,65000000,2012,2.35,99,1375,65000000,6.4
